In [27]:
import pandas as pd
import matplotlib.pyplot as plt

# AIF360 Libraries
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing
from aif360.algorithms.postprocessing import  CalibratedEqOddsPostprocessing

# Scikit-learn Libraries
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report
import seaborn as sns

# Load the Law School dataset
def load_law_dataset():
    df = pd.read_csv('../../data/law_school_clean.csv')
    
    # Print column info to inspect the dataset structure
    print("Column names:", df.columns.tolist())
    print("Preview of dataframe:")
    print(df.head(2))
    
    # Based on the snippets shown, the second to last column seems to be 'race'
    # and the last column appears to be a binary outcome (likely what we need)
    protected_attribute = 'race'  # Confirm this is the correct column name
    
    # Use the last column as the target - update this after seeing the actual columns
    class_label = df.columns[-1]  
    print(f"Using '{class_label}' as the target variable")
    
    majority_group_name = "White"
    minority_group_name = "Non-White"
    
    # Label encode categorical variables
    le = preprocessing.LabelEncoder()
    for i in df.columns:
        if df[i].dtypes == 'object':
            df[i] = le.fit_transform(df[i])
    
    return df, protected_attribute, majority_group_name, minority_group_name, class_label

In [28]:
# Load and prepare dataset
df, protected_attribute, majority_group, minority_group, class_label = load_law_dataset()

# Convert to AIF360 dataset
dataset = StandardDataset(
    df=df,
    label_name=class_label,
    favorable_classes=[1],
    protected_attribute_names=[protected_attribute],
    privileged_classes=[[1]]  # Assuming 1 indicates majority group (White)
)

# Define privileged and unprivileged groups
privileged_groups = [{protected_attribute: 1}]
unprivileged_groups = [{protected_attribute: 0}]

# Split data into train, validation, and test sets
train, vt = dataset.split([0.6], shuffle=True)
validation, test = vt.split([0.5], shuffle=True)

# Calculate initial fairness metrics
original_metrics = BinaryLabelDatasetMetric(
    dataset=train,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

print("Initial Fairness Metrics:")
print(f"Statistical Parity Difference: {original_metrics.statistical_parity_difference():.4f}")
print(f"Disparate Impact: {original_metrics.disparate_impact():.4f}")

Column names: ['decile1b', 'decile3', 'lsat', 'ugpa', 'zfygpa', 'zgpa', 'fulltime', 'fam_inc', 'male', 'tier', 'race', 'pass_bar']
Preview of dataframe:
   decile1b  decile3  lsat  ugpa  zfygpa  zgpa  fulltime  fam_inc  male  tier  \
0      10.0     10.0  44.0   3.5    1.33  1.88       1.0      5.0   0.0   4.0   
1       5.0      4.0  29.0   3.5   -0.11 -0.57       1.0      4.0   0.0   2.0   

    race  pass_bar  
0  White       1.0  
1  White       1.0  
Using 'pass_bar' as the target variable
Initial Fairness Metrics:
Statistical Parity Difference: -0.2022
Disparate Impact: 0.7804


In [32]:
# Function to extract features and labels
def get_features_labels(dataset):
    X = dataset.features
    y = dataset.labels.ravel()
    return X, y

# Get training and test data
X_train, y_train = get_features_labels(train)
X_test, y_test = get_features_labels(test)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train baseline model (LogisticRegression)
baseline_model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)
baseline_model.fit(X_train_scaled, y_train)
y_pred = baseline_model.predict(X_test_scaled)

# Evaluate baseline model performance
print("\nBaseline Model Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

# Create dataset with predictions
test_pred = test.copy()
test_pred.labels = y_pred.reshape(-1, 1)

# Calculate classification metrics for fairness
class_metrics = ClassificationMetric(
    dataset=test,
    classified_dataset=test_pred,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

# Print fairness metrics for the baseline model
print("\nFairness Metrics for Baseline Model:")
print(f"Equal Opportunity Difference: {class_metrics.equal_opportunity_difference():.4f}")
print(f"Average Odds Difference: {class_metrics.average_odds_difference():.4f}")
print(f"Disparate Impact: {class_metrics.disparate_impact():.4f}")
print(f"Statistical Parity Difference: {class_metrics.statistical_parity_difference():.4f}")


Baseline Model Performance:
Accuracy: 0.9022
Balanced Accuracy: 0.6146
              precision    recall  f1-score   support

         0.0       0.63      0.25      0.35       454
         1.0       0.91      0.98      0.95      3706

    accuracy                           0.90      4160
   macro avg       0.77      0.61      0.65      4160
weighted avg       0.88      0.90      0.88      4160


Fairness Metrics for Baseline Model:
Equal Opportunity Difference: -0.0838
Average Odds Difference: -0.2072
Disparate Impact: 0.8263
Statistical Parity Difference: -0.1710


In [33]:
# Fixed evaluation framework to properly assess on test data
def fairness_integrity_framework(train_dataset, test_dataset, model, protected_attr, privileged_groups, unprivileged_groups):
    """
    An improved framework for evaluating model fairness and integrity that properly separates
    training and testing data
    """
    # Extract features and labels for training
    X_train = train_dataset.features
    y_train = train_dataset.labels.ravel()
    
    # Extract features and labels for testing
    X_test = test_dataset.features
    y_test = test_dataset.labels.ravel()
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train model on training data
    model.fit(X_train_scaled, y_train)
    
    # Evaluate on test data
    y_pred = model.predict(X_test_scaled)
    
    # Create dataset with predictions
    pred_dataset = test_dataset.copy()
    pred_dataset.labels = y_pred.reshape(-1, 1)
    
    # Calculate fairness metrics on test data
    dataset_metrics = BinaryLabelDatasetMetric(
        dataset=test_dataset,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )
    
    classification_metrics = ClassificationMetric(
        dataset=test_dataset,
        classified_dataset=pred_dataset,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )
    
    # Compile results into a fairness report
    fairness_report = {
        "data_metrics": {
            "statistical_parity_difference": dataset_metrics.statistical_parity_difference(),
            "disparate_impact": dataset_metrics.disparate_impact(),
            "class_imbalance": dataset_metrics.num_positives(privileged=True) / dataset_metrics.num_instances(privileged=True) - 
                              dataset_metrics.num_positives(privileged=False) / dataset_metrics.num_instances(privileged=False)
        },
        "classification_metrics": {
            "accuracy": accuracy_score(y_test, y_pred),
            "balanced_accuracy": balanced_accuracy_score(y_test, y_pred),
            "average_odds_difference": classification_metrics.average_odds_difference(),
            "disparate_impact": classification_metrics.disparate_impact(),
            "statistical_parity_difference": classification_metrics.statistical_parity_difference(),
            "equal_opportunity_difference": classification_metrics.equal_opportunity_difference(),
            "theil_index": classification_metrics.theil_index()
        }
    }
    
    return fairness_report

# Apply the framework to our baseline model - FIXED to match new function signature
baseline_integrity_report = fairness_integrity_framework(
    train_dataset=train,
    test_dataset=test,  # Now properly passing test dataset
    model=baseline_model,
    protected_attr=protected_attribute,
    privileged_groups=privileged_groups,
    unprivileged_groups=unprivileged_groups
)

# Display the framework results
print("\nFairness Integrity Framework Results:")
for category, metrics in baseline_integrity_report.items():
    print(f"\n{category.upper()}:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")


Fairness Integrity Framework Results:

DATA_METRICS:
  statistical_parity_difference: -0.1901
  disparate_impact: 0.7937
  class_imbalance: 0.1901

CLASSIFICATION_METRICS:
  accuracy: 0.9022
  balanced_accuracy: 0.6146
  average_odds_difference: -0.2072
  disparate_impact: 0.8263
  statistical_parity_difference: -0.1710
  equal_opportunity_difference: -0.0838
  theil_index: 0.0424


In [34]:
# 1. Pre-processing technique: Reweighing
print("\nApplying Reweighing technique...")
reweighing = Reweighing(unprivileged_groups=unprivileged_groups,
                      privileged_groups=privileged_groups)
train_reweighed = reweighing.fit_transform(train)

# 2. Pre-processing technique: Disparate Impact Remover
print("\nApplying Disparate Impact Remover...")
di_remover = DisparateImpactRemover(repair_level=0.8)
train_di_removed = di_remover.fit_transform(train)
test_di_removed = di_remover.fit_transform(test)

# 3. Post-processing technique: Calibrated Equalized Odds
# Train model on original data
print("\nPreparing for Calibrated Equalized Odds...")
lr_model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)
X_train_orig, y_train_orig = get_features_labels(train)
X_validation, y_validation = get_features_labels(validation)

# Scale features
scaler = StandardScaler()
X_train_orig_scaled = scaler.fit_transform(X_train_orig)
X_validation_scaled = scaler.transform(X_validation)

# Train model
lr_model.fit(X_train_orig_scaled, y_train_orig)

# Get predictions and scores on validation set for calibration
validation_pred = validation.copy()
validation_pred.scores = lr_model.predict_proba(X_validation_scaled)[:,1].reshape(-1,1)
validation_pred.labels = lr_model.predict(X_validation_scaled).reshape(-1,1)

# Apply calibrated equalized odds
print("\nApplying Calibrated Equalized Odds...")
eq_odds = CalibratedEqOddsPostprocessing(
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)
eq_odds.fit(validation, validation_pred)

# Apply post-processing to test data
X_test, y_test = get_features_labels(test)
X_test_scaled = scaler.transform(X_test)
test_pred = test.copy()
test_pred.scores = lr_model.predict_proba(X_test_scaled)[:,1].reshape(-1,1)
test_pred.labels = lr_model.predict(X_test_scaled).reshape(-1,1)
test_pred_eq_odds = eq_odds.predict(test_pred)

# 4. Alternative model: Random Forest
print("\nTraining Random Forest model...")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)


Applying Reweighing technique...

Applying Disparate Impact Remover...

Preparing for Calibrated Equalized Odds...

Applying Calibrated Equalized Odds...

Training Random Forest model...


In [35]:
# Dictionary of models to evaluate - properly separating train and test data for consistent evaluation
models_to_evaluate = {
    "Baseline": (train, test, LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)),
    "Reweighing": (train_reweighed, test, LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)),
    "Disparate Impact Remover": (train_di_removed, test_di_removed, LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)),
    "Random Forest": (train, test, RandomForestClassifier(n_estimators=100, random_state=42))
}

# Dictionary to store results
fairness_results = {}

# Evaluate each technique
for name, (train_dataset, test_dataset, model) in models_to_evaluate.items():
    print(f"\nEvaluating {name}...")
    fairness_results[name] = fairness_integrity_framework(
        train_dataset=train_dataset,
        test_dataset=test_dataset,
        model=model,
        protected_attr=protected_attribute,
        privileged_groups=privileged_groups,
        unprivileged_groups=unprivileged_groups
    )
    
# Evaluate post-processed model separately
print("\nEvaluating Calibrated Equalized Odds...")
# Extract metrics from test_pred_eq_odds
eq_odds_metrics = ClassificationMetric(
    dataset=test,
    classified_dataset=test_pred_eq_odds,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
)

# Handle the special case of Calibrated EqOdds
fairness_results["Calibrated EqOdds"] = {
    "classification_metrics": {
        "accuracy": accuracy_score(y_test, test_pred_eq_odds.labels.ravel()),
        "balanced_accuracy": balanced_accuracy_score(y_test, test_pred_eq_odds.labels.ravel()),
        "average_odds_difference": eq_odds_metrics.average_odds_difference(),
        "disparate_impact": eq_odds_metrics.disparate_impact(),
        "statistical_parity_difference": eq_odds_metrics.statistical_parity_difference(),
        "equal_opportunity_difference": eq_odds_metrics.equal_opportunity_difference(),
        "theil_index": eq_odds_metrics.theil_index()
    }
}


Evaluating Baseline...

Evaluating Reweighing...

Evaluating Disparate Impact Remover...

Evaluating Random Forest...

Evaluating Calibrated Equalized Odds...


In [36]:
# Function to create a comparative table of fairness metrics
def compare_fairness_metrics(results_dict):
    metrics_to_compare = [
        "accuracy",
        "balanced_accuracy",
        "statistical_parity_difference",
        "disparate_impact",
        "equal_opportunity_difference",
        "average_odds_difference",
        "theil_index"
    ]
    
    # Create a dataframe for comparison
    comparison_data = []
    for technique, results in results_dict.items():
        row = {"Technique": technique}
        
        # Add metrics - handle cases where data_metrics might not exist
        for metric in metrics_to_compare:
            if "classification_metrics" in results and metric in results["classification_metrics"]:
                row[metric] = results["classification_metrics"][metric]
        
        comparison_data.append(row)
    
    return pd.DataFrame(comparison_data)

# Create and display the comparison table
fairness_comparison = compare_fairness_metrics(fairness_results)
print("\nComparative Fairness Metrics:")
print(fairness_comparison.round(4))

# Visualize the results with multiple plots
plt.figure(figsize=(15, 10))

# Create multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle('Comparison of Fairness Metrics Across Techniques', fontsize=16)

# 1. Accuracy vs Statistical Parity Difference
ax = axes[0, 0]
for technique in fairness_comparison['Technique']:
    ax.scatter(
        fairness_comparison.loc[fairness_comparison['Technique']==technique, 'accuracy'],
        fairness_comparison.loc[fairness_comparison['Technique']==technique, 'statistical_parity_difference'],
        label=technique,
        s=100
    )
ax.axhline(y=0, color='grey', linestyle='--', alpha=0.5)
ax.set_xlabel('Accuracy')
ax.set_ylabel('Statistical Parity Difference')
ax.grid(True, linestyle='--', alpha=0.5)
ax.set_title('Accuracy vs. Statistical Parity')
for i, technique in enumerate(fairness_comparison['Technique']):
    ax.annotate(technique, 
               (fairness_comparison.loc[fairness_comparison['Technique']==technique, 'accuracy'].iloc[0],
                fairness_comparison.loc[fairness_comparison['Technique']==technique, 'statistical_parity_difference'].iloc[0]),
               xytext=(5, 5), textcoords='offset points')

# 2. Accuracy vs Equal Opportunity Difference
ax = axes[0, 1]
for technique in fairness_comparison['Technique']:
    ax.scatter(
        fairness_comparison.loc[fairness_comparison['Technique']==technique, 'accuracy'],
        fairness_comparison.loc[fairness_comparison['Technique']==technique, 'equal_opportunity_difference'],
        label=technique,
        s=100
    )
ax.axhline(y=0, color='grey', linestyle='--', alpha=0.5)
ax.set_xlabel('Accuracy')
ax.set_ylabel('Equal Opportunity Difference')
ax.grid(True, linestyle='--', alpha=0.5)
ax.set_title('Accuracy vs. Equal Opportunity')
for i, technique in enumerate(fairness_comparison['Technique']):
    ax.annotate(technique, 
               (fairness_comparison.loc[fairness_comparison['Technique']==technique, 'accuracy'].iloc[0],
                fairness_comparison.loc[fairness_comparison['Technique']==technique, 'equal_opportunity_difference'].iloc[0]),
               xytext=(5, 5), textcoords='offset points')

# 3. Bar chart of fairness metrics
ax = axes[1, 0]
metrics = ['statistical_parity_difference', 'equal_opportunity_difference', 'average_odds_difference']
subset_df = fairness_comparison[['Technique'] + metrics].melt(
    id_vars=['Technique'], 
    value_vars=metrics, 
    var_name='Metric', 
    value_name='Value'
)
sns.barplot(x='Technique', y='Value', hue='Metric', data=subset_df, ax=ax)
ax.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax.set_title('Fairness Metrics by Technique')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# 4. Bar chart of disparate impact
ax = axes[1, 1]
disparate_impact = fairness_comparison[['Technique', 'disparate_impact']]
sns.barplot(x='Technique', y='disparate_impact', data=disparate_impact, ax=ax)
ax.axhline(y=1, color='red', linestyle='-', alpha=0.3)
ax.set_title('Disparate Impact by Technique')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
ax.set_ylim([min(0.5, disparate_impact['disparate_impact'].min() - 0.1), 
             max(1.5, disparate_impact['disparate_impact'].max() + 0.1)])

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.savefig('../visualizations/fairness_comparison_plots.png', dpi=300, bbox_inches='tight')
plt.show()


Comparative Fairness Metrics:
                  Technique  accuracy  balanced_accuracy  \
0                  Baseline    0.9022             0.6146   
1                Reweighing    0.9022             0.6146   
2  Disparate Impact Remover    0.9026             0.6071   
3             Random Forest    0.8925             0.6111   
4         Calibrated EqOdds    0.8995             0.5822   

   statistical_parity_difference  disparate_impact  \
0                        -0.1710            0.8263   
1                        -0.1710            0.8263   
2                        -0.1797            0.8185   
3                        -0.1745            0.8210   
4                        -0.1862            0.8138   

   equal_opportunity_difference  average_odds_difference  theil_index  
0                       -0.0838                  -0.2072       0.0424  
1                       -0.0838                  -0.2072       0.0424  
2                       -0.0869                  -0.2325       0.04

/tmp/ipykernel_60905/2724488431.py:91: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
/tmp/ipykernel_60905/2724488431.py:99: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
/tmp/ipykernel_60905/2724488431.py:106: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [37]:
# Extract key metrics for the assessment
baseline_spd = fairness_results['Baseline']['classification_metrics']['statistical_parity_difference']
baseline_eod = fairness_results['Baseline']['classification_metrics']['equal_opportunity_difference']
baseline_acc = fairness_results['Baseline']['classification_metrics']['accuracy']

# Find best performing technique for fairness
techniques = [tech for tech in fairness_results.keys() if tech != 'Baseline']
fairness_improvements = {}

for tech in techniques:
    # Calculate improvements in fairness metrics (reduction in absolute values)
    spd_improvement = abs(baseline_spd) - abs(fairness_results[tech]['classification_metrics']['statistical_parity_difference'])
    eod_improvement = abs(baseline_eod) - abs(fairness_results[tech]['classification_metrics']['equal_opportunity_difference'])
    
    # Normalize by baseline values to get percentage improvement
    spd_pct = spd_improvement / abs(baseline_spd) if baseline_spd != 0 else spd_improvement
    eod_pct = eod_improvement / abs(baseline_eod) if baseline_eod != 0 else eod_improvement
    
    # Overall fairness improvement score (equal weight to both metrics)
    fairness_improvements[tech] = 0.5 * spd_pct + 0.5 * eod_pct

# Find best technique for fairness
best_fairness_technique = max(fairness_improvements.items(), key=lambda x: x[1])[0] if fairness_improvements else None

# Find technique with best accuracy
accuracy_by_technique = {tech: results['classification_metrics']['accuracy'] 
                        for tech, results in fairness_results.items()}
best_accuracy_technique = max(accuracy_by_technique.items(), key=lambda x: x[1])[0]

# Generate critical assessment
assessment = f"""
# Critical Assessment of Fairness Mitigation Techniques

## Dataset Characteristics and Inherent Biases
- The Law School dataset shows a significant **disparate impact** of {fairness_results['Baseline']['classification_metrics']['disparate_impact']:.4f} for the baseline model.
- Statistical parity difference of {baseline_spd:.4f} indicates that White students have a higher probability of passing the bar exam.
- Equal opportunity difference of {baseline_eod:.4f} suggests disparities in true positive rates across racial groups.

## Effectiveness of Mitigation Techniques

### Pre-processing Techniques
- **Reweighing**: {
    "Improved fairness metrics while maintaining accuracy" if fairness_results['Reweighing']['classification_metrics']['accuracy'] >= baseline_acc * 0.95 else
    "Improved fairness metrics but with noticeable accuracy trade-offs"
}
- **Disparate Impact Remover**: {
    "Successfully reduced disparate impact" if abs(fairness_results['Disparate Impact Remover']['classification_metrics']['disparate_impact'] - 1) < abs(baseline_spd - 1) else
    "Limited effectiveness in reducing disparate impact"
}

### Post-processing Techniques
- **Calibrated Equalized Odds**: {
    "Effectively balanced error rates across groups" if abs(fairness_results['Calibrated EqOdds']['classification_metrics']['equal_opportunity_difference']) < abs(baseline_eod) else
    "Limited effectiveness in balancing error rates"
}

### Alternative Models
- **Random Forest**: {
    "Offered better fairness-accuracy balance than baseline logistic regression" if (
        fairness_results['Random Forest']['classification_metrics']['accuracy'] >= baseline_acc and
        abs(fairness_results['Random Forest']['classification_metrics']['statistical_parity_difference']) <= abs(baseline_spd)
    ) else
    "Did not substantially improve the fairness-accuracy trade-off"
}

## Key Trade-offs and Findings
1. **Accuracy vs. Fairness Trade-off**: {"Most" if sum(1 for tech in fairness_improvements if fairness_improvements[tech] > 0 and accuracy_by_technique[tech] < baseline_acc) > len(techniques)/2 else "Some"} fairness interventions resulted in accuracy reductions.
   
2. **Best Overall Approach**: **{best_fairness_technique}** provided the best fairness improvements, while **{best_accuracy_technique}** maintained the highest accuracy.

3. **Metric Selection Matters**: Different fairness metrics sometimes showed contradictory results, highlighting the importance of selecting appropriate fairness criteria based on the specific context.

## Limitations
1. **Binary Protected Attribute**: Our analysis treated race as a binary attribute (White/Non-White), which oversimplifies the complex nature of racial identity.
   
2. **Dataset Limitations**: The Law School dataset is historical and may not reflect current demographic patterns or educational practices.

3. **Limited Context**: Technical fairness metrics cannot fully capture the societal and institutional factors contributing to disparities in bar exam passage rates.

## Recommendations for Practical Implementation
1. Deploy a combination of pre-processing and post-processing methods to address biases at multiple stages of the ML pipeline.
   
2. Integrate regular fairness audits into model maintenance processes.
   
3. Supplement technical mitigations with policy interventions addressing the root causes of educational disparities.

4. Consider intersectional fairness to account for individuals belonging to multiple disadvantaged groups.

## Conclusion
While algorithmic fairness interventions demonstrated meaningful improvements in bias metrics, achieving true fairness in predicting law school outcomes requires a holistic approach that combines technical solutions with institutional and societal change.
"""

print(assessment)


# Critical Assessment of Fairness Mitigation Techniques

## Dataset Characteristics and Inherent Biases
- The Law School dataset shows a significant **disparate impact** of 0.8263 for the baseline model.
- Statistical parity difference of -0.1710 indicates that White students have a higher probability of passing the bar exam.
- Equal opportunity difference of -0.0838 suggests disparities in true positive rates across racial groups.

## Effectiveness of Mitigation Techniques

### Pre-processing Techniques
- **Reweighing**: Improved fairness metrics while maintaining accuracy
- **Disparate Impact Remover**: Successfully reduced disparate impact

### Post-processing Techniques
- **Calibrated Equalized Odds**: Limited effectiveness in balancing error rates

### Alternative Models
- **Random Forest**: Did not substantially improve the fairness-accuracy trade-off

## Key Trade-offs and Findings
1. **Accuracy vs. Fairness Trade-off**: Some fairness interventions resulted in accuracy reductio

In [38]:
# Apply the most promising technique to test data
best_technique = best_fairness_technique if fairness_improvements[best_fairness_technique] > 0 else 'Baseline'
print(f"\nApplying best technique ({best_technique}) to test data...")

if best_technique == "Reweighing":
    # Apply reweighing to test data
    X_test, y_test = get_features_labels(test)
    X_test_scaled = scaler.transform(X_test)
    
    # Train model on reweighted training data
    reweigh_model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)
    X_train_reweigh, y_train_reweigh = get_features_labels(train_reweighed)
    X_train_reweigh_scaled = scaler.fit_transform(X_train_reweigh)
    reweigh_model.fit(X_train_reweigh_scaled, y_train_reweigh)
    
    # Predict on test data
    y_pred_reweigh = reweigh_model.predict(X_test_scaled)
    test_reweigh_pred = test.copy()
    test_reweigh_pred.labels = y_pred_reweigh.reshape(-1, 1)
    
    # Calculate final metrics
    final_metrics = ClassificationMetric(
        dataset=test,
        classified_dataset=test_reweigh_pred,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )
    
elif best_technique == "Disparate Impact Remover":
    # Apply disparate impact remover to test data
    X_test_di, y_test_di = get_features_labels(test_di_removed)
    X_test_di_scaled = scaler.transform(X_test_di)
    
    # Train model on transformed training data
    di_model = LogisticRegression(solver='liblinear', max_iter=1000, random_state=42)
    X_train_di, y_train_di = get_features_labels(train_di_removed)
    X_train_di_scaled = scaler.fit_transform(X_train_di)
    di_model.fit(X_train_di_scaled, y_train_di)
    
    # Predict on test data
    y_pred_di = di_model.predict(X_test_di_scaled)
    test_di_pred = test_di_removed.copy()
    test_di_pred.labels = y_pred_di.reshape(-1, 1)
    
    # Calculate final metrics
    final_metrics = ClassificationMetric(
        dataset=test_di_removed,
        classified_dataset=test_di_pred,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )

elif best_technique == "Calibrated EqOdds":
    # Already calculated above
    final_metrics = eq_odds_metrics

elif best_technique == "Random Forest":
    # Apply random forest to test data
    X_test, y_test = get_features_labels(test)
    X_test_scaled = scaler.transform(X_test)
    
    # Train model
    rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    X_train_scaled, y_train = get_features_labels(train)
    X_train_scaled = scaler.fit_transform(X_train_scaled)
    rf_model.fit(X_train_scaled, y_train)
    
    # Predict on test data
    y_pred_rf = rf_model.predict(X_test_scaled)
    test_rf_pred = test.copy()
    test_rf_pred.labels = y_pred_rf.reshape(-1, 1)
    
    # Calculate final metrics
    final_metrics = ClassificationMetric(
        dataset=test,
        classified_dataset=test_rf_pred,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )

else:  # Default to baseline
    X_test, y_test = get_features_labels(test)
    X_test_scaled = scaler.transform(X_test)
    
    # Predict on test data
    y_pred = baseline_model.predict(X_test_scaled)
    test_pred = test.copy()
    test_pred.labels = y_pred.reshape(-1, 1)
    
    # Calculate final metrics
    final_metrics = class_metrics

# Print final fairness assessment
print("\nFinal Fairness Assessment on Test Data:")
print(f"Statistical Parity Difference: {final_metrics.statistical_parity_difference():.4f}")
print(f"Disparate Impact: {final_metrics.disparate_impact():.4f}")
print(f"Equal Opportunity Difference: {final_metrics.equal_opportunity_difference():.4f}")
print(f"Average Odds Difference: {final_metrics.average_odds_difference():.4f}")
print(f"Theil Index: {final_metrics.theil_index():.4f}")


Applying best technique (Baseline) to test data...

Final Fairness Assessment on Test Data:
Statistical Parity Difference: -0.1710
Disparate Impact: 0.8263
Equal Opportunity Difference: -0.0838
Average Odds Difference: -0.2072
Theil Index: 0.0424
